In [2]:
import tensorflow as tf
import numpy as np
import Reading

In [3]:
epoch = 160
batch_size = 10

def one_hot(labels,Label_class):
    one_hot_label = np.array([[int(i == int(labels[j])) for i in range(Label_class)] for j in range(len(labels))])   
    return one_hot_label

#定義初始化變數 採用normal distribution , 標準差為0.02
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.02)
    return tf.Variable(initial)
#定義初始化變數 採用常數 , 皆為0.1
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#convolution layer
def conv2d(x,W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

#max_pool layer
def max_pool_4x4(x):
    return tf.nn.max_pool(x, ksize=[1,4,4,1], strides=[1,4,4,1], padding='SAME')
# 定義placeholder
x = tf.placeholder(tf.float32, [batch_size,128,128,3])
y_ = tf.placeholder(tf.float32, [batch_size,4])

In [4]:
#first convolution and max_pool layer
W_conv1 = weight_variable([5,5,3,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_4x4(h_conv1)

#second convolution and max_pool layer
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_4x4(h_conv2)

#變成全連接層，用一個mlp處理
reshape = tf.reshape(h_pool2,[batch_size, -1])
dim = reshape.get_shape()[1].value
W_fc1 = weight_variable([dim, 1024])
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(reshape, W_fc1) + b_fc1)

#dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024,4])
b_fc2 = bias_variable([4])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
#損失函數和演算法

# 計算最小交叉熵
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))

train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
# 計算預測正確率
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:
Use tf.cast instead.


In [6]:
# 自己做好的 TF 檔在哪裡，自己知道
filename = 'C:/Users/USER/Desktop/tfrecord/Train.tfrecords'
testname = 'C:/Users/USER/Desktop/tfrecord/Test.tfrecords'
# 調用剛才的函數
image, label = Reading.read_and_decode(filename)
img_test, label_test = Reading.read_and_decode(testname)
#使用shuffle_batch可以隨意打亂輸入
img_batch, label_batch = tf.train.shuffle_batch([image, label],
                                                batch_size=batch_size, 
                                                capacity=400,
                                                min_after_dequeue=100)

img_test, label_test = tf.train.shuffle_batch([img_test, label_test],
                                                batch_size=batch_size,
                                                capacity=400,
                                                min_after_dequeue=100)


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecor

In [7]:
with tf.Session() as sess:
    # 初始化是必要的動作
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    # 建立執行緒協調器
    coord = tf.train.Coordinator()
    # 啟動文件隊列，開始讀取文件
    threads=tf.train.start_queue_runners(coord=coord) 
    
    #batch_idxs = int(2314/batch_size)
    #for i in range(1,epoch+1):
        #for j in range(1,batch_size+1):
            #val, l = sess.run([img_batch, label_batch])
            #l = one_hot(l,2)
            #_, acc = sess.run([train_step, accuracy], feed_dict={x: val, y_: l, keep_prob: 0.5})
            #if i % 2 == 0:
                #print("Epoch:[%4d] [%4d/%4d], accuracy:[%.8f]" % (i, j, batch_size, acc) )
                
    for k in range(1,epoch+1):
        val, l = sess.run([img_batch, label_batch])
        l = one_hot(l,4)
        
         # 送資料進去訓練
        sess.run(train_step , feed_dict={x: val, y_: l, keep_prob: 0.5})
        if k % 10 == 0:
            acc = sess.run(accuracy, feed_dict={x: val, y_: l, keep_prob: 0.5})
            print("Epoch:[%4d] , accuracy:[%.8f]" % (k, acc) )           
    
    #測試資料
    val, l = sess.run([img_test, label_test])
    l = one_hot(l,4)
    print(l)
    y, acc = sess.run([y_conv,accuracy], feed_dict={x: val, y_: l, keep_prob: 1})
    
    print("test accuracy: [%.8f]" % (acc))
    
    #驗證模型
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x:val , y_: l}))

    coord.request_stop()
    coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Epoch:[  10] , accuracy:[0.40000001]
Epoch:[  20] , accuracy:[0.69999999]
Epoch:[  30] , accuracy:[1.00000000]
Epoch:[  40] , accuracy:[1.00000000]
Epoch:[  50] , accuracy:[1.00000000]
Epoch:[  60] , accuracy:[1.00000000]
Epoch:[  70] , accuracy:[1.00000000]
Epoch:[  80] , accuracy:[1.00000000]
Epoch:[  90] , accuracy:[1.00000000]
Epoch:[ 100] , accuracy:[1.00000000]
Epoch:[ 110] , accuracy:[1.00000000]
Epoch:[ 120] , accuracy:[1.00000000]
Epoch:[ 130] , accuracy:[1.00000000]
Epoch:[ 140] , accuracy:[1.00000000]
Epoch:[ 150] , accuracy:[1.00000000]
Epoch:[ 160] , accuracy:[1.00000000]
[[1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 1 0 0]]
test accuracy: [1.00000000]
1.0
